## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Merging Dataframes

1. 학생 정보 관련 파일:
   - studentInfo.csv: 학생의 인구 통계 정보, 등록 정보 등을 담고 있습니다.

2. 과정 정보 관련 파일:
   - courses.csv: 각 코스(모듈)에 대한 정보를 담고 있습니다.
   - assessments.csv: 코스 내의 평가 정보 (과제, 시험 등)를 담고 있습니다.

3. 학생 성적 관련 파일:
    - studentAssessment.csv: 학생들의 평가 점수를 담고 있습니다.
    - studentRegistration.csv: 학생의 코스 등록 정보를 담고 있습니다.
    
4. VLE 활동 관련 파일:
   - vle.csv: VLE 활동에 대한 설명 정보를 담고 있습니다.
   - studentVle.csv: 학생의 VLE(Virtual Learning Environment) 활동 데이터를 담고 있습니다.

In [7]:
assessments_df = pd.read_csv("./data/assessments.csv")
courses_df = pd.read_csv("./data/courses.csv")
studentAssessment_df = pd.read_csv("./data/studentAssessment.csv")

# vle_df = pd.read_csv("./data/vle.csv")
# studentVle_df = pd.read_csv("./data/studentVle.csv")

studentInfo_df = pd.read_csv("./data/studentInfo.csv")
studentRegistration_df = pd.read_csv("./data/studentRegistration.csv")

In [ ]:
print('코스 내의 과제, 시험 정보',assessments_df.columns)
print('코스 정보',courses_df.columns)
print('학생 평가 점수',studentAssessment_df.columns)
print('학생 정보',studentInfo_df.columns)
print('학생의 등록 정보',studentRegistration_df.columns)
# print('vle 활동 정보',vle_df.columns)
# print('학생의 vle 활동 정보',studentVle_df.columns)

In [9]:
# 1. 학생 평가 점수와 과제 정보 병합
merged_df = studentAssessment_df.merge(assessments_df, on='id_assessment', how='left')
# 2. 학생 정보 추가
merged_df = merged_df.merge(studentInfo_df, on=['id_student', 'code_module', 'code_presentation'], how='left')
# 3. 학생 등록 정보 추가
merged_df = merged_df.merge(studentRegistration_df, on=['id_student', 'code_module', 'code_presentation'], how='left')
# 4. 코스 정보 추가
merged_df = merged_df.merge(courses_df, on=['code_module', 'code_presentation'], how='left')

In [10]:
# 중복 행 제거
merged_df = merged_df.drop_duplicates()

In [11]:
merged_df.to_csv('merged_data.csv')

In [12]:
# merged_vle_df = pd.merge(vle_df, studentVle_df, 
#                            on=['id_site', 'code_module', 'code_presentation'],
#                            how='outer')

In [13]:
# 중복 행 제거
# merged_vle_df = merged_vle_df.drop_duplicates()

In [14]:
# merged_vle_df.to_csv('./data/merged_vle_data.csv')

---
# 데이터 전처리

### 1. 결측치 처리

### 2. 이상치 처리 및 시각화
   - 수치형 변수의 이상치 확인 및 처리(예: score, studied_credits의 극단값 혹은 0 값 등)

### 3. 데이터 타입 변환
   - 날짜 관련 변수를 datetime 형식으로 변환
   - 범주형 변수를 category 타입으로 변환

### 4. 특성 엔지니어링

In [15]:
merged_data=merged_df

#### 학생의 성적 관련 특성
  - 각 학생의 평균 점수, 최고 점수, 최저 점수, 점수의 표준편차
    - 각 학생당 코스별 성적 편차 필요할까?
  - 점수 추세 (상승 또는 하락)
  - 과제 난이도에 따른 가중 점수 -> 난이도 기준을 뭘로 잡아야하나?
 
##### my_average_score, my_max_score, my_min_score, my_score_std, my_score_trend, assesment_weight, weighted_score

- my_avg_score : 개인 학업 성취도 수준 파악
- my_max/min_score/my_score_std : 특정 과목 강점/약점 식별 및 극단적 편차 분석
- my_score_trend	: 학습 효과성 평가 (지속적 상승=효율적 학습법, 하락=개입 필요)
- weighted_score :	난이도 대비 성취도 → "B과제는 고난이도지만 고가중점수 → Distinct 학생" 

In [16]:
# 학생의 평균, 최고, 최저 점수 및 표준편차 계산
student_scores = merged_data.groupby('id_student')['score'].agg(['mean', 'max', 'min', 'std']).reset_index()
student_scores.columns = ['id_student', 'my_average_score', 'my_max_score', 'my_min_score', 'my_score_std']

# # 학생의 코스별 성적 표준편차 계산
# score_std_by_module = merged_data.groupby(['id_student', 'code_module'])['score'].std().reset_index()
# score_std_by_module = score_std_by_module.pivot(index='id_student', columns='code_module', values='score').reset_index()
# score_std_by_module.columns = ['id_student'] + [f'score_std_{col}' for col in score_std_by_module.columns if col != 'id_student']

# 점수 추세 분석
merged_data_sorted = merged_data.sort_values(by=['id_student', 'date_submitted'])
merged_data_sorted['score_diff'] = merged_data_sorted.groupby('id_student')['score'].diff()

def determine_trend(diff):
    if diff > 0:
        return 'increasing'
    else:
        return 'decreasing'
    
merged_data_sorted['score_trend'] = merged_data_sorted['score_diff'].apply(determine_trend)
student_trends = merged_data_sorted.groupby('id_student')['score_trend'].apply(lambda x: x.value_counts().idxmax()).reset_index()
student_trends.columns = ['id_student', 'my_score_trend']

# 모든 특성을 하나의 데이터프레임으로 병합
student_features = pd.merge(student_scores, student_trends, on='id_student')

# 원본 데이터와 새로운 특성을 병합
merged_data = pd.merge(merged_data, student_features, on='id_student')

# 과제 난이도 가중치를 적용한 점수 (평균 기반)

# weight 계산 (평균 점수가 낮을수록 가중치가 높아짐)
avg = merged_data.groupby('id_assessment')['score'].mean()
max_avg = avg.max()
weight = (max_avg - avg) / (max_avg - avg).sum()

# weight를 데이터프레임으로 변환
weight_df = weight.reset_index()
weight_df.columns = ['id_assessment', 'assessment_weight']
merged_data = pd.merge(merged_data, weight_df, on='id_assessment', how='left')

# 가중 점수 계산
merged_data['weighted_score'] = merged_data['score'] * merged_data['assessment_weight']

#### 코스 관련 특성
  - 코스별 평균 점수, 최고 점수, 최저 점수, 점수의 표준편차 -> 최저 점수 다 0임, 최고점이 필요한가?, 중간값이 필요한가?
  - 코스별 과제 개수

##### 'course_avg_score', 'course_std_score', 'assignment_count'

- course_avg_score, course_std_score :	평가 방식 적정성 → 편차↑=과도한 변별력 / 강좌 난이도 레벨링
- assignment_count :	과제량-성적 상관관계 분석 → 최적 과제량 도출

In [17]:
# 과목별 통계 계산
subject_stats = merged_data.groupby(['code_presentation', 'code_module']).agg({
    'score': ['mean', 'max', 'std']
})
subject_stats.columns = ['course_avg_score', 'course_max_score', 'course_std_score']

# 과목 통계 데이터 병합
merged_data = merged_data.merge(
    subject_stats.reset_index(),
    on=['code_presentation', 'code_module'],
    how='left'
)
merged_data['assignment_count'] = merged_data.groupby(['code_presentation', 'code_module'])['id_assessment'].transform('nunique')

# # 과목별 난이도 가중치를 적용한 점수 (표준편차 기반)
# difficulty = subject_stats['course_std_score']
# weights = (difficulty / difficulty.sum()) * 10
# merged_data = merged_data.merge(
#     weights.rename('course_weight'), 
#     left_on='code_module', 
#     right_index=True
# )
# merged_data['weighted_score'] = merged_data['score'] * merged_data['course_weight']

In [ ]:
merged_data[merged_data['code_module']=='AAA']

#### 행동 패턴 특성
- 전체 학생 기준
  - 코스별 과제 제출률 => 제출률 100프로
  - 코스별 지각 제출 비율
  - 과제별 제출 시간의 평균, 중앙값, 표준편차

- 한명 학생 기준
  - 과제 제출률 => 제출률 100프로
  - 지각 제출 비율
  - 이탈 학생의 이탈 단계 할당

##### 'course_late_rate', 'my_late_rate', 'days_early_submission_avg', 'days_early_submissione_max', 'days_early_submission_min','days_early_submission_std','days_early_submission', 'is_late', 'unregistration_stage'

- my_late_rate	: 습관적 지각 여부 → 학습태도 평가 지표
- days_early_submission_*	: 과제의 난의도 판별 → 평균 7일 이상 = 쉬운 과제인가, 평균 0일 이하 = 어려운 과제인가, 표준편차가 크다 = 학생의 역량
- course_late_rate : 코스의 난의도 판별 -> 필요없을까?
- unregistration_stage : 첫 번째 과제 이후 이탈-> 수업의 어려움보다는 외부적인 이유일 가능성이 높음 / 중간쯤 과제 제출 후 이탈->학업에서의 고충이나 과제의 난이도 등이 영향이 있을 수 있음

In [19]:
# 모든 학생 기준
# 1. 코스별 평균 제출률 계산
# 1-1. 학생별 코스 내 제출률 계산
# student_course_submission = merged_data.groupby(['code_module', 'id_student']).agg(
#     student_submissions=('id_assessment', 'count'),
#     course_assessments=('id_assessment', 'nunique')
# ).astype({'student_submissions': int, 'course_assessments': int}).reset_index()

# # 1-2. 학생별 제출률 계산
# student_course_submission['student_submission_rate'] = (
#     student_course_submission['student_submissions']
#     / student_course_submission['course_assessments']
# )
# course_stats = student_course_submission.groupby('code_module')['student_submission_rate'].mean().reset_index()
# course_stats.columns = ['code_module', 'course_avg_submission_rate']
# student_course_submission[student_course_submission['student_submission_rate'] != 1]

# 2. 코스별 지각 제출 비율
# 3. 과제별 제출시간 평균, 중간, 표준편차

# 개인기준
# # 1. 학생별 과제 제출률
# student_stats = merged_data.groupby('id_student').agg(
#     student_submissions=('id_assessment', 'count'),
#     total_assignments=('id_assessment', 'nunique')
# ).reset_index()
# student_stats['student_submission_rate'] = student_stats['student_submissions'] / student_stats['total_assignments']
# student_stats[student_stats['student_submission_rate'] != 1]

# 2. 학생별 지각 제출 비율

In [20]:
# 코스별 지각 제출 비율
merged_data['is_late'] = (merged_data['date_submitted'] > merged_data['date']).astype(int)
merged_data['course_late_rate'] = merged_data.groupby(['code_presentation', 'code_module'])['is_late'].transform('mean')

# 과제별 데드라인 전 제출 시간 평균, 중간, 표준편차
merged_data['days_early_submission'] = merged_data['date'] - merged_data['date_submitted']  # 컬럼명 변경 권장
assessment_stats = merged_data.groupby('id_assessment')['days_early_submission'].agg([
    ('days_early_submission_avg', 'mean'),
    ('days_early_submission_max', 'max'),
    ('days_early_submission_min', 'min'),
    ('days_early_submission_std', 'std')
]).reset_index()
merged_data = merged_data.merge(assessment_stats, on='id_assessment')

# 학생별 지각 제출 비율
merged_data['my_late_rate'] = merged_data.groupby('id_student')['is_late'].transform('mean')

In [21]:
dropped_out_students = merged_data[merged_data['date_unregistration'].notnull()]

# 이탈한 학생이 제출한 과제 수 계산 (학생, 과목별로 그룹화)
submitted_assignments = dropped_out_students.groupby(['id_student', 'code_presentation', 'code_module'])['id_assessment'].count().reset_index()
submitted_assignments.rename(columns={'id_assessment': 'submitted_count'}, inplace=True)
merged_data = merged_data.merge(submitted_assignments, on=['id_student', 'code_presentation', 'code_module'], how='left')

# 이탈 학생의 과제 제출률 계산 (이탈 학생의 과목당 제출 과제 수 / 과목당 전체 과제 수)
merged_data['submitted_proportion'] = merged_data['submitted_count'] / merged_data['assignment_count']

# 이탈 학생의 이탈 단계 할당
def assign_stage_based_on_proportion(row):
    if pd.isna(row['date_unregistration']):
        return np.nan
    elif row['submitted_proportion'] < 0.33:  # Less than 33% of total assignments
        return 1
    elif row['submitted_proportion'] < 0.66:  # Between 33% and 66%
        return 2
    else:                                     # More than 66%
        return 3

merged_data['unregistration_stage'] = merged_data.apply(assign_stage_based_on_proportion, axis=1)


In [ ]:
merged_data.loc[merged_data['unregistration_stage'].notnull(), 
                ['id_student','code_presentation', 'code_module','unregistration_stage', 'submitted_count', 'assignment_count', 'date_unregistration']]


#### 아래부터는 EDA 아이디어가 정해진 후 진행

### 5. 불필요한 특성 제거
   - 분석에 불필요하거나 중복되는 정보를 가진 열 제거

### 6. 범주형 변수 인코딩
   - 필요에 따라 원-핫 인코딩 또는 라벨 인코딩 적용

### 7. 스케일링(-> 모델링 시점에 할 지 안할지 정해도 될듯)
   - 수치형 변수에 대해 표준화 또는 정규화 적용

---
# **Exploratory Data Analysis**

## **특성간의 상관관계 확인**
- 특성간의 상관관계 확인 후 상관계수가 높은 변수끼리 그룹화하여 그룹별 성과와의 상관관계 분석

In [ ]:
merged_data.info()
merged_data.columns

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df=merged_data
# 상관 행렬 계산
matrix = df.drop(columns=['my_min_score','my_max_score','is_banked', 'id_assessment', 'id_student','score','is_late','days_early_submission_max','days_early_submission_min','days_early_submission_std','days_early_submission_avg'])
corr_matrix = matrix.corr(numeric_only=True)

# 상삼각 행렬 마스킹
upper_mask = np.triu_indices_from(corr_matrix, k=1)
high_corr_values = corr_matrix.values[upper_mask]
average_corr = high_corr_values.mean()

# 상/하위 필터링 (상삼각만 적용)
high_corr = corr_matrix.where(
    (corr_matrix > average_corr) & 
    np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
)
low_corr = corr_matrix.where(
    (corr_matrix < average_corr) & 
    np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
)

# 상위 20개 추출
top_20 = high_corr.unstack().dropna().sort_values(ascending=False).head(20)
down_20 = low_corr.unstack().dropna().sort_values().head(20)

# 변수 목록 생성 (집합 → 리스트 변환)
def get_unique_vars(pairs):
    return list({var for pair in pairs.index for var in pair})

vars_to_plot = get_unique_vars(top_20)
vars_to_plot2 = get_unique_vars(down_20)

# 히트맵 생성
def plot_heatmap(data, title):
    plt.figure(figsize=(12,10))
    sns.heatmap(data, annot=True, cmap='coolwarm', fmt=".2f", 
                center=0, vmin=-1, vmax=1)
    plt.title(title)
    plt.show()

plot_heatmap(corr_matrix.loc[vars_to_plot, vars_to_plot], 
            "Top 20 Highest Correlations")
plot_heatmap(corr_matrix.loc[vars_to_plot2, vars_to_plot2], 
            "Bottom 20 Lowest Correlations")


### 여기부터는 위의 상관관계 확인후 분석 들어가야함

## (예시)**온라인 학습 활동 분석**
- sum_click(총 클릭 수)와 final_result(최종 성적) 간 상관관계 분석
- 학습 콘텐츠 유형(activity_type)별 평균 클릭 수(sum_click) 비교
- week_from, week_to를 활용한 주차별 학습 패턴 분석 (lineplot)

## (예시)**학생의 연령, 교육 수준, 장애 여부 등이 학습 성과와 관련 있는지 분석**
- age_band, highest_education, disability 등과 final_result 간 분포 비교 (countplot)
- num_of_prev_attempts(재수강 횟수)와 final_result 간 상관관계 분석
- date_unregistration 값이 있는 경우, 조기 이탈 학생과 학습 활동 비교